In [0]:
import keras
import time

In [0]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  

In [0]:
(X_pretrain, y_pretrain), (X_pretest, y_pretest) = keras.datasets.mnist.load_data()

In [0]:
(X, y), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
y_true = y_test

In [0]:
minValue = np.min(X)
maxValue = np.max(X)
X = 2 * ((X - minValue) / (maxValue - minValue) - 0.5)
X_test = 2 * ((X_test - minValue) / (maxValue - minValue) - 0.5)

In [0]:
X.shape, X_test.shape

In [0]:
X = np.dstack([X] * 3)
X_test = np.dstack([X_test]*3)
X.shape, X_test.shape

In [0]:
X = X.reshape(-1, 28,28,3)
X_test= X_test.reshape (-1,28,28,3)
X.shape, X_test.shape

In [0]:
## Изменяем размер на 32х32, в сете есть такой вариант, можно 150х150, но моя тачка не выдерживает таких объемов и облако тоже не дает
from keras.preprocessing.image import img_to_array, array_to_img

X = np.asarray([img_to_array(array_to_img(im, scale=False).resize((32,32))) for im in X])
X_test = np.asarray([img_to_array(array_to_img(im, scale=False).resize((32,32))) for im in X_test])

#Поэтому так
X.shape, X_test.shape

In [0]:
y = keras.utils.to_categorical(y, num_classes=10)
y_test = keras.utils.to_categorical(y_test, num_classes=10)
y.shape,y_test.shape

In [0]:
from sklearn.model_selection import train_test_split

#Разобьем 20/80 тренировочный сет
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=13)

# получим 2 сета вот такого размера
X_train.shape, X_val.shape, y_train.shape, y_val.shape

In [0]:
from keras.applications import VGG19

# Инициализируем модель извне
vgg19 = VGG19(weights=None , include_top=False, input_shape = (32, 32, 3), classes = 10)
vgg19.summary()

In [0]:
from keras.applications.vgg19 import preprocess_input
 
X_train = preprocess_input(X_train)
X_val = preprocess_input(X_val)
X_test = preprocess_input(X_test)

In [0]:

# Запустите сверточную сеть по набору данных
train_features = vgg19.predict(np.array(X_train), batch_size=256, verbose=1)
test_features = vgg19.predict(np.array(X_test), batch_size=256, verbose=1)
val_features = vgg19.predict(np.array(X_val), batch_size=256, verbose=1)

In [0]:
#  Размеры весов
print(train_features.shape, "\n",  test_features.shape, "\n", val_features.shape)

In [0]:
# Flatten extracted features
train_features = np.reshape(train_features, (48000, 1*1*512))
test_features = np.reshape(test_features, (10000, 1*1*512))
val_features = np.reshape(val_features, (12000, 1*1*512))

**1. Модель без изменений, но с полным ее переобучением**




In [0]:
from keras.layers import Dense, Dropout
from keras.models import Model
from keras import models
from keras import layers
from keras import optimizers
from keras.layers import Dense, Dropout, Flatten


model = models.Sequential()
model.add(layers.Dense(10, activation="softmax"))


In [0]:
import keras
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [0]:

train_features.shape, y_train.shape

In [0]:
y_val.shape,val_features.shape

In [0]:
start_time = time.clock()
history = model.fit(train_features, y_train,
          batch_size=256,
          epochs=50,
          verbose=1,
          validation_data=(val_features, y_val))
fit_time = time.clock() - start_time
print("Fit time: %f", fit_time)



In [0]:
start_time = time.clock()
score = model.evaluate(test_features, y_test, verbose=0)
fit_time = time.clock() - start_time
print("Eval one image time: %f", fit_time/y_test.shape[0])

In [0]:
score = model.evaluate(test_features, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [0]:

import matplotlib.pyplot as plt
%matplotlib inline

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.title('Точность на обучающем множестве')
plt.plot(epochs, acc, 'red', label='Трен. точность')
plt.plot(epochs, val_acc, 'blue', label='Тест. точность')
plt.legend()

plt.figure()
plt.title('Функция потерь')
plt.plot(epochs, loss, 'red', label='Трен')
plt.plot(epochs, val_loss, 'blue', label='Тест')

plt.legend()

plt.show()

In [0]:
predicted_classes = model.predict_classes(test_features)
predicted_classes.shape

In [0]:
print(y_true)
print(y_pretest)

In [0]:
correct = np.nonzero(predicted_classes==y_true)[0]
incorrect = np.nonzero(predicted_classes!=y_true)[0]

In [0]:
from sklearn.metrics import classification_report
target_names = ["Класс {}".format(i) for i in range(10)]
print(classification_report(y_true, predicted_classes, target_names=target_names))

**2.	Замена классификатора в исходной модели**

In [0]:
from keras.layers import Dense, Dropout
from keras.models import Model
from keras import models
from keras import layers
from keras import optimizers

# Добавляем поверх готовых весов еще один полносвязанный и дропаут
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_dim=1 * 1 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation="softmax"))

In [0]:
import keras
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [0]:
start_time = time.clock()
history = model.fit(train_features, y_train,
          batch_size=256,
          epochs=50,
          verbose=1,
          validation_data=(val_features, y_val))
fit_time = time.clock() - start_time
print("Fit time: %f", fit_time)

start_time = time.clock()
score = model.evaluate(test_features, y_test, verbose=0)
fit_time = time.clock() - start_time
print("Eval one image time: %f", fit_time/X_test.shape[0])

In [0]:
score = model.evaluate(test_features, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [0]:

import matplotlib.pyplot as plt
%matplotlib inline

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.title('Точность на обучающем множестве')
plt.plot(epochs, acc, 'red', label='Трен. точность')
plt.plot(epochs, val_acc, 'blue', label='Тест. точность')
plt.legend()

plt.figure()
plt.title('Функция потерь')
plt.plot(epochs, loss, 'red', label='Трен')
plt.plot(epochs, val_loss, 'blue', label='Тест')

plt.legend()

plt.show()

In [0]:
correct = np.nonzero(predicted_classes==y_true)[0]
incorrect = np.nonzero(predicted_classes!=y_true)[0]
from sklearn.metrics import classification_report
target_names = ["Класс {}".format(i) for i in range(10)]
print(classification_report(y_true, predicted_classes, target_names=target_names))

In [0]:
print(classification_report(y_test, predicted_classes, target_names=target_names))